# Environment Setup
Please check the [wandbのreport](https://api.wandb.ai/links/wandb-healthcare/jm7scchv) for details on setting up the environment.


## Weights and Biases setup
The progress and charts of model training can be visualized through Weights and Biases (wandb). Please refer to the above report for instructions on setting up wandb. Below, you will set up the entity and project, which are the storage locations for wandb. Please enter any values you wish.

In [ ]:
import os
#os.environ["WANDB_ENTITY"]="wandb-healthcare"
os.environ["WANDB_PROJECT"]="BioNeMo_Molecule_LLM"

## Confirmation of GPU

In [ ]:
!nvidia-smi

## Confirmation of GPU

In [ ]:
!nvidia-smi

# Data Preprocessing

We will use a very small subset of the original ZINC15 dataset, provided as a sample dataset to briefly introduce the model training functionality of the BioNeMo Framework. This subset will be saved in /workspace/bionemo/examples/molecule/molmim/data by executing the following command.

For this test run, the folder contains /train, /val, and /test subfolders, each containing molecular structures in SMILES format as CSV files. Check how many files have been created in these directories and adjust the TRAIN_FILE_RANGE, VAL_FILE_RANGE, and TEST_FILE_RANGE parameters in the example command below accordingly. The current command setup is configured to handle one file in each of the train, val, and test directories, containing 299069, 2, and 1486 samples respectively

In [ ]:
cd /workspace/bionemo

In [ ]:
!python examples/molecule/molmim/pretrain.py\
 --config-path conf\
 --config-name molmim_70m_24_3\
 ++do_training=False\
 ++trainer.devices=1 \
 ++trainer.num_nodes=1 \
 ++model.data.dataset_path=/workspace/bionemo/examples/molecule/molmim/data \
 ++model.data.links_file=/workspace/bionemo/examples/molecule/megamolbart/dataset/ZINC-downloader-sample.txt \
 ++exp_manager.create_wandb_logger=False

In [ ]:
"""
import wandb
with wandb.init(name="data_upload") as run:
    artifact = wandb.Artifact(
        name="small_ZINC",
        type="dataset",
        description="subset of ZINC",
        metadata={"path":"/workspace/bionemo/examples/molecule/molmim/data"},
    )
    artifact.add_dir("/workspace/bionemo/examples/molecule/molmim/data")
    run.log_artifact(artifact)
"""

# Pretraining

For this test run, we will use the pre-configured parameters provided in the pretrain_small_canonicalized_logv.yaml configuration file located in the /workspace/bionemo/examples/molecule/molmim/conf folder. This configuration was used to train the MolMIM-70M-24.3 checkpoint

In [ ]:
cd /workspace/bionemo 

Please set the TRAIN_FILE_RANGE, TEST_FILE_RANGE, and VAL_FILE_RANGE according to the size of the data

In [ ]:
#TRAIN_FILE_RANGE="x_OP_000..175_CL_"
#TEST_FILE_RANGE="x_OP_000..004_CL_"
#VAL_FILE_RANGE="x_OP_000..175_CL_"

TRAIN_FILE_RANGE="x000"
TEST_FILE_RANGE="x000"
VAL_FILE_RANGE="x000"

In [ ]:
import os
os.environ["HYDRA_FULL_ERROR"]="1"

In [ ]:
!python examples/molecule/molmim/pretrain.py\
 --config-path conf\
 --config-name molmim_70m_24_3\
 ++trainer.devices=1 \
 ++trainer.num_nodes=1 \
 ++trainer.max_steps=1000 \
 ++trainer.accumulate_grad_batches=1 \
 ++trainer.val_check_interval=5 \
 ++trainer.limit_val_batches=1.0 \
 ++trainer.precision=32 \
 ++model.micro_batch_size=128 \
 ++model.global_batch_size=128 \
 ++model.dwnstr_task_validation.enabled=False \
 ++model.data.dataset_path=examples/molecule/molmim/data \
 ++model.data.dataset.train=$TRAIN_FILE_RANGE \
 ++model.data.dataset.val=$VAL_FILE_RANGE \
 ++model.data.dataset.test=$TEST_FILE_RANGE \
 ++model.data.index_mapping_dir=/results/data_index/ \
 ++model.seq_length=128 \
 ++exp_manager.exp_dir=/results/logs/ \
 ++exp_manager.create_wandb_logger=False

#  Property-guided molecular optimization using MolMIM with CMA-ES

This session demonstrates how to load a MolMIM checkpoint from the BioNeMo Framework, optimize several molecules of interest using a custom user-defined scoring function, and select novel related molecules expected to improve performance as measured by the scoring function using CMA-ES to traverse the latent space of the MolMIM model. To sample these molecules, the following steps must be completed:

1. Load the desired MolMIM checkpoint.

2. Encode the starting molecule into the latent space of MolMIM.

3. Execute CMA-ES, repeating the following:
* Decode the latent representations into SMILES strings.
* Apply the user-defined scoring function to these SMILES strings, generating pairs of SMILES/scores.
* Request a new set of latent space representations from the CMA-ES algorithm.

Next, we will download the pretrained model. To download the model, you need to install ngc and set up ngc config.

In [ ]:
import os
#os.environ["WANDB_ENTITY"]="wandb-healthcare"
os.environ["WANDB_PROJECT"]="BioNeMo_Molecure_optimization"

In [ ]:
!pip install optuna statsmodels -qqq

In [ ]:
!wget -q -O /tmp/ngccli_linux.zip --content-disposition https://api.ngc.nvidia.com/v2/resources/nvidia/ngc-apps/ngc_cli/versions/3.38.0/files/ngccli_linux.zip && unzip -o /tmp/ngccli_linux.zip -d /tmp && chmod u+x /tmp/ngc-cli/ngc && rm /tmp/ngccli_linux.zip

Next, to download a pretrained model, you will need to install NGC (NVIDIA GPU Cloud) and configure your NGC settings. Here’s how you can proceed:

/tmp/ngc-cli/ngc config set

/tmp/ngc-cli/ngc config set

To complete the configuration of your NGC CLI, follow these steps, inputting the requested details:

`API Key`: Enter your API key from the NVIDIA GPU Cloud (NGC). This key is critical for authentication and accessing the services.

`CLI Output Format`: You can choose the desired output format for your CLI responses. If unsure, the default format is usually fine, so just press "Enter".

`Organization (Org)`: Choose an organization other than 'no-org'. If you are part of an NVIDIA-approved organization, input its name; otherwise, consult NGC documentation or your NGC account settings for possible values.

`Team`: If your NGC usage involves a specific team setup, enter the team name. If not, just press "Enter" to skip this step.

`ACE`: This is typically for advanced configuration settings related to application, compute, and environment. Press "Enter" to accept default settings unless specific modifications are required.

After configuring NGC, you can download the model using the command below.

最後に、下記のコマンドを入力すれば、モデルをダウンロードできます。

In [ ]:
cd /workspace/bionemo 

In [ ]:
!python download_models.py --download_dir /workspace/bionemo/models molmim_70m_24_3

## Load the checkpoint into the molmim inference wrapper

In [ ]:
from bionemo.utils.hydra import load_model_config
import os
from bionemo.model.molecule.molmim.infer import MolMIMInference
bionemo_home=f"/workspace/bionemo"
os.environ['BIONEMO_HOME'] = bionemo_home
checkpoint_path = f"{bionemo_home}/models/molecule/molmim/molmim_70m_24_3.nemo"
cfg = load_model_config(config_name="molmim_infer.yaml", config_path=f"{bionemo_home}/examples/tests/conf/") # reasonable starting config for molmim inference
# This is the field of the config that we need to set to our desired checkpoint path.
cfg.model.downstream_task.restore_from_path = checkpoint_path
model = MolMIMInference(cfg, interactive=True)

## Setting Up User-Defined Molecular Scoring Functions
In this section, users can incorporate their own scoring functions that they want to optimize. In this example, we optimize a combination of Tanimoto similarity and Quantitative Estimate of Drug-likeness (QED) with the input molecule. This follows the example from the first MolMIM paper:

<h3><center>score=min (QED/0.9, 1) + min (Tanimoto/0.4, 1)</center></h3>
In this case, the model is allowed to optimize up to a maximum of QED 0.9 and Tanimoto similarity 0.4. Once these maximums are achieved, no further optimization is performed.

In [ ]:
from typing import List, Optional

import numpy as np

from guided_molecule_gen.oracles import qed, tanimoto_similarity

def score_mixing_function(qeds, similarities):
    # We want to maximize QED and tanimoto similarity up to 0.9 and 0.4, respectively.
    return np.clip(qeds / 0.9, a_min=0.0, a_max=1.0) + np.clip(similarities / 0.4, a_min=0.0, a_max=1.0)

def try_canon(smiles:str) -> Optional[str]:
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smiles), canonical=True)
    except:
        return None

def canonicalize(smiles: List[str]) -> List[str]:
    return [try_canon(s) for s in smiles]


def scoring_function(smiles: List[str], reference:str, **kwargs) -> np.ndarray:
    """Takes a list of SMILES strings and returns an array of scores.

    Args:
        smiles (List[str]): Smiles strings to generate a score for (one each)
        reference (str): Reference molecule (SMILES string) is also used for this scoring function.

    Returns:
        np.ndarray: Array of scores, one for each input SMILES string.
    """
    #csmiles = canonicalize(smiles)
    scores: np.ndarray = score_mixing_function(qed(smiles), tanimoto_similarity(smiles, reference))
    return -1 * scores

## Definition of Starting Molecules
In this section, we define the starting molecules for the optimization process. As examples, we will use imatinib, erlotinib, and gefitinib. Ensure that the SMILES strings representing these molecules are canonicalized using RDKit. Since MolMIM is trained on a corpus of SMILES strings canonicalized with RDKit, both the input and output should also be canonicalized with RDKit to achieve the best performance

In [ ]:
import wandb
from rdkit import Chem
from rdkit.Chem.QED import qed as rdkit_qed

# SMILES strings and compound names
compound_names = ["Imatinib", "Erlotinib", "Gifitinib"]
starting_smiles = [
    "CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5", # Imatinib
    "COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC", # Erlotinib
    "C1COCCN1CCCOc2c(OC)cc3ncnc(c3c2)Nc4cc(Cl)c(F)cc4", # Gifitinib
]

# Generate RDKit molecules from SMILES
molecules = [Chem.MolFromSmiles(smile) for smile in starting_smiles]

# Calculate QED scores
starting_qed = [rdkit_qed(mol) for mol in molecules]

# Generate canonical SMILES
canonicalized_smiles = [Chem.MolToSmiles(mol, canonical=True) for mol in molecules]

# Initialize a W&B run
with wandb.init(name="simple-EDA") as run:

    # Create a W&B Table
    table = wandb.Table(columns=["Compound Name", "SMILES", "Canonical SMILES", "Structure"])

    # Add data to the table
    for name, smiles, canonical, mol in zip(compound_names, starting_smiles, canonicalized_smiles, molecules):
        wandb_molecule = wandb.Molecule.from_rdkit(mol)
        table.add_data(name, smiles, canonical, wandb_molecule)

    # Log the table to W&B
    run.log({"Chemical Structures": table})

## Configuring the Optimizer and Wrapping the Inference API for CMA-ES
The CMA-ES library expects the input/output of the inference model to be in a specific format. Therefore, we provide a wrapper for this, and below we demonstrate how to set up the optimization.

In [ ]:
from bionemo.model.core.controlled_generation import ControlledGenerationPerceiverEncoderInferenceWrapper

controlled_gen_kwargs = {
    "sampling_method": "beam-search",
    "sampling_kwarg_overrides": {"beam_size": 3, "keep_only_best_tokens": True, "return_scores": False},
}

model_wrapped = ControlledGenerationPerceiverEncoderInferenceWrapper(
    model, enforce_perceiver=True, hidden_steps=1, **controlled_gen_kwargs
)  # just flatten the position for this.

## CMA-ES tuning
Different models require different optimal settings for CMA-ES. Here, we perform a grid search over possible values of sigma and then carry out further optimization steps with the best settings. To optimize this sigma hyperparameter, we use the Optuna library. This process is called Hyperparameter Optimization (HPO).
Let's look at the HPO process with Optuna and the optimal sigma values found using wandb.

In [ ]:
import wandb
from guided_molecule_gen.optimizer import MoleculeGenerationOptimizer
import optuna
from datetime import datetime


def objective(trial, n_steps:int=10):
    sigma = trial.suggest_float('sigma', 0, 2)
    optimizer = MoleculeGenerationOptimizer(
        model_wrapped,
        scoring_function,
        canonicalized_smiles,
        popsize=10,  # larger values will be slower but more thorough
        optimizer_args={"sigma": sigma},
    )

    optimizer.optimize(n_steps)
    final_smiles = optimizer.generated_smis
    final_score = np.mean([np.min(scoring_function(smis_population, reference_smis)) for smis_population,reference_smis in zip(final_smiles, canonicalized_smiles)])
    wandb.log({"score": final_score, "sigma": sigma})
    return final_score

with wandb.init(name=f"Basic_Optuna_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
    study = optuna.create_study()
    study.optimize(objective, n_trials=50)
    # After the study, log the best parameters and the DataFrame
    best_params = study.best_params
    wandb.log({"best_params": best_params})

    # Get a DataFrame of all trials and save it as a W&B Table
    df = study.trials_dataframe()
    wandb_table = wandb.Table(dataframe=df)
    wandb.log({"optuna_history_table": wandb_table})

上記の値はHPOプロセスで得られた最適値ですが、有効な値の範囲を考慮し、よりロバストであろう最小値を選択します。HPOプロセスは確率的なため、高性能な値と低性能な値が近接して存在することがあります。オプティマイザが一般的に良好に機能するシグマ値の適切な範囲を特定したいと考えています。滑らかにした最適な選択と、最も良い名目上の選択をwandb上で比較しましょう

In [ ]:
import wandb
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from guided_molecule_gen.optimizer import MoleculeGenerationOptimizer
from datetime import datetime


def lowess_with_confidence_bounds(x, y, eval_x, N=200, conf_interval=0.95, lowess_kw=None):
    """
    Perform Lowess regression and determine a confidence interval by bootstrap resampling.
    """
    # Lowess smoothing
    smoothed = sm.nonparametric.lowess(exog=x, endog=y, xvals=eval_x, **lowess_kw)

    # Bootstrap resampling for confidence intervals
    smoothed_values = np.empty((N, len(eval_x)))
    for i in range(N):
        sample = np.random.choice(len(x), len(x), replace=True)
        sampled_x = x[sample]
        sampled_y = y[sample]
        smoothed_values[i, :] = sm.nonparametric.lowess(exog=sampled_x, endog=sampled_y, xvals=eval_x, **lowess_kw)
        

    # Confidence intervals
    sorted_values = np.sort(smoothed_values, axis=0)
    bound = int(N * (1 - conf_interval) / 2)
    bottom = sorted_values[bound - 1]
    top = sorted_values[-bound]

    return smoothed, bottom, top


# Initialize a W&B run
with wandb.init(name=f"HPO_visualization_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
    # Generate data, fit and plot
    completed_trials = [trial for trial in study.trials if trial.state == optuna.trial.TrialState.COMPLETE]
    trials_data = [{"sigma": trial.params["sigma"], "loss": trial.value, "trial_id": tid} for tid,trial in enumerate(completed_trials)]
    data = pd.DataFrame(trials_data)
    data = pd.DataFrame(trials_data)
    eval_x = np.linspace(0, 2, 200)
    smoothed, bottom, top = lowess_with_confidence_bounds(data.sigma, data.loss, eval_x, lowess_kw={"frac": 0.33})

    # Create plot

    fig, ax = plt.subplots()
    ax.scatter(data["sigma"], data["loss"], label="Observed Points")
    ax.plot(eval_x, smoothed, 'k', label="LOWESS Smoothed")
    ax.fill_between(eval_x, bottom, top, color='blue', alpha=0.5, label="95% Confidence Interval")
    ax.legend()
    ax.set_title("Loss vs Sigma with LOWESS Fit")
    
    # Log the plot to W&B
    run.log({"LOWESS Plot": wandb.Image(fig)})

    # Additional data to log
    smoothed_best_sigma = eval_x[np.argmin(smoothed)]  # Use the smoothed minimum
    run.log({"best_params": study.best_params, "smoothed_best_sigma": smoothed_best_sigma})
    smooth_best = {"sigma": smoothed_best_sigma}
    print("smooth_best:", smooth_best, "simple_best_param:",study.best_params)

## CMA-ES Optimization
### Conducting a Larger-Scale CMA-ES Optimization with the Discovered Parameters
Since the sigma values found through HPO have proven effective, we increase the size of the molecular population and the number of steps to finally perform a larger-scale optimization.

### Exploring the Results
To assess the performance of the optimization, we can quantify the number of invalid samples produced. "Invalid" refers to SMILES strings that do not represent chemically viable molecules.
After the run, we will create graphs showing how the components of the target (QED and Tanimoto similarity) changed in each iteration. According to the definition of the target, values of Tanimoto similarity above 0.4 are considered optimal, so noise around this value is expected. Similarly, for QED, values above 0.9 are considered optimal, so if there are molecules exceeding this threshold, noise around these values is also expected.
Let's check the results in WandB.

### How Good Was the Optimization Performance?
To evaluate the performance of the optimization, we can quantify the number of invalid samples produced. "Invalid" refers to SMILES strings that do not represent chemically viable molecules.

-> Check num_of_bad_samples in wandb

Ultimately, we can quantify the degree of QED improvement relative to baseline values and the percentage of optimized molecules that maintained the desired Tanimoto similarity threshold of above 0.4.

-> Check mean_qed_improvement and tanimoto_above_04 in wandb.


In [ ]:
import wandb
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from guided_molecule_gen.optimizer import MoleculeGenerationOptimizer
from tqdm import trange
from datetime import datetime

# Initialize a W&B run
with wandb.init(name=f"CMS_EA_Optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
    # Assuming model_wrapped and scoring_function are defined elsewhere
    optimizer = MoleculeGenerationOptimizer(
        model_wrapped,
        scoring_function,
        canonicalized_smiles,
        popsize=50,  # larger values will be slower but more thorough
        optimizer_args=smooth_best,  # Vals from HPO
    )
    
    columns = ["iteration", "idx","molecure", "smiles", "structure", "qud_score", "tanimoto_score"]
    optimization_results = wandb.Table(columns=columns)

    # Starting state for idx 0
    qed_scores = [qed(canonicalized_smiles)]
    tanimoto_scores = [[tanimoto_similarity([canonicalized_smiles[idx]], canonicalized_smiles[idx])[0] for idx in range(len(canonicalized_smiles))]]
    best_molecules = [canonicalized_smiles]
    fraction_bad_samples = [[0] * len(canonicalized_smiles)]

    for i in trange(30):
        optimizer.step()
        final_smiles = optimizer.generated_smis
        # Population of molecules is returned, but we only want the best one.
        _qed_scores = []
        _tanimoto_scores = []
        _best_molecules = []
        _fraction_bad = []
        ca_id = 0
        for smis_population, reference_smis in zip(final_smiles, canonicalized_smiles):
            idx = np.argmin(scoring_function(smis_population, reference_smis))
            _fraction_bad.append(np.mean(qed(smis_population) == 0))
            _best_molecules.append(smis_population[idx])
            _qed_scores.append(qed([smis_population[idx]])[0])
            _tanimoto_scores.append(tanimoto_similarity([smis_population[idx]], reference_smis)[0])
            
            
            mol = Chem.MolFromSmiles(smis_population[idx])
            wandb_molecule = wandb.Molecule.from_rdkit(mol)
            
            optimization_results.add_data(i, idx,["imatinib", "erlotinib", "gifitinib"][ca_id], smis_population[idx], wandb_molecule, qed([smis_population[idx]])[0], tanimoto_similarity([smis_population[idx]], reference_smis)[0])
            ca_id=+1

        qed_scores.append(_qed_scores)
        tanimoto_scores.append(_tanimoto_scores)
        best_molecules.append(_best_molecules)
        fraction_bad_samples.append(_fraction_bad)

    run.log({"optimization_results":optimization_results})

    # Plotting results
    fig, ax = plt.subplots()
    for i, molecule in enumerate(["imatinib", "erlotinib", "gifitinib"]):
        line, = plt.plot(np.arange(len(qed_scores)), [q[i] for q in qed_scores], label=f"{molecule} QED")
        color = line.get_color()
        plt.plot(np.arange(len(tanimoto_scores)), [t[i] for t in tanimoto_scores], label=f"{molecule} Tanimoto", linestyle="--", color=color)
    plt.axhline(y=0.9, color='r', linestyle='-', label="QED target")
    plt.axhline(y=0.4, color='r', linestyle='--', label="Tanimoto target")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel("Iteration")
    plt.ylabel("QED or Tanimoto similarity")
    plt.title("Targets over time for MolMIM model")
    plt.tight_layout()

    # Log the plot to W&B
    run.log({"LOWESS Plot": wandb.Image(fig)})

    # Additional calculations and logs
    qed_improvements = []
    tanimoto_above_04 = []
    for i in range(len(starting_qed)):
        tanimoto_above_04.append(tanimoto_scores[-1][i] >= 0.4)
        qed_improvements.append(qed_scores[-1][i] - starting_qed[i])
    
    metrics = {
        "num_of_bad_samples": np.mean(fraction_bad_samples),
        "mean_qed_improvement": np.mean(qed_improvements),
        "tanimoto_above_04": np.mean(tanimoto_above_04),
    }
    
    run.log(metrics)
    
    # Print metrics
    print("num_of_bad_samples:", np.mean(fraction_bad_samples),"mean_qed_improvement:", metrics["mean_qed_improvement"], "tanimoto_above_04:", metrics["tanimoto_above_04"])
